In [1]:
import numpy as np
import pandas as pd
import os
import vlc
import cv2
import time
import ctypes
import platform
import threading
import keyboard
import subprocess
import matplotlib.pyplot as plt
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from tkVideoPlayer import TkinterVideo
from tkvideo import tkvideo
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel
from PyQt5.QtGui import QPixmap
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

In [4]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [5]:
from tensorflow.keras.models import load_model

# Load the pre-trained CNN model
model.load_weights('vgg_face_weights.h5')
vgg = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [6]:
def make_vgg(img1):
    img1 = cv2.resize(img1, (224, 224))
    img1 = np.array(img1).reshape(-1, 224, 224, 3)
    img1 = img1 / 255
    img1_representation = vgg.predict(img1, verbose=0)
    return img1_representation

In [7]:
def compare(img1, img2):
    euclidean_distance = np.linalg.norm(img1[0] - img2[0])
    if euclidean_distance < 0.45:
        return True
    else:
        return False

In [8]:
def compare2(img1, img2):
    euclidean_distance = np.linalg.norm(img1[0] - img2[0])
    return euclidean_distance

In [9]:
def format_duration(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60

    duration = "{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds)
    return duration

In [10]:
def count_frame():
    global count
    global canvas
    color = (0, 0, 255)
    canvas = np.full((300, 1920, 3), color, np.uint8)
    try:
        cv2.putText(canvas, "Number of Smiles: " + str(count), (10, 50), 0, 1, (255,255,255), 2)
    except:
        count = count

In [11]:
def play_video():
    
    global ad_video
    global off
    global face_color
    global face_smile
    global canvas
    global count
    
    off = False
    
    while (off == False):
    # Create a VideoCapture object
        ad_video = cv2.VideoCapture('Ads/Ads1.mp4')
        frames = ad_video.get(cv2.CAP_PROP_FRAME_COUNT)
        fps = ad_video.get(cv2.CAP_PROP_FPS)
        seconds = frames / fps
        sleep_time = int(950/fps)

        while True:
            t1 = time.time()
            # Read a frame from the video
            ret, frame = ad_video.read()
#             frame = cv2.resize(frame, (1920, 1080))

            if not ret:
                ad_video = cv2.VideoCapture('Ads/Ads1.mp4')
                ret, frame = ad_video.read()
                
            frame = cv2.resize(frame, (1920, 1080))
            
            try:         
#                 frame[0:300, 0:1920] = canvas
                cv2.putText(frame, "Number of smiles: " + str(count), (300, 100), 0, 1, (0, 0, 0), 2)
                face_smile = cv2.resize(face_smile, (300,300))
                frame[0:300, 1290:1590] = face_smile
                print("Frame Merged.")
        
            except:
                off = False
                print("except")
                
            # Display the frame
            cv2.namedWindow('AD', cv2.WINDOW_NORMAL)
            cv2.setWindowProperty('AD', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
            cv2.imshow('AD', frame)

            sleep_needed = int(sleep_time-((time.time()-t1)*1000))
            if sleep_needed < 1:
                sleep_needed = 1

            # Check for 'q' key press to exit
            key = cv2.waitKey(sleep_needed)

            if key == 27:
                off = True
                break

    # Release the VideoCapture object and close the window
    ad_video.release()
    cv2.destroyAllWindows()

In [12]:
threading.Thread(target=play_video).start()
faces_dict = {}
start_time = time.time()
cap = cv2.VideoCapture(0)
count = 0

while True:
    _ , img = cap.read()
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_rect = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=15)
    
    elapsed_time = int(time.time() - start_time)
#     cv2.putText(img, "{} seconds".format(elapsed_time), (10, 100), 0, 1, (255,255,255), 2)
    
    for (x,y,w,h) in face_rect:
#         cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), thickness = 2)
        face_gray = gray_img[y:y + h, x:x + w]
        face_color = img[y:y + h, x:x + w]
        smile_rect = smile_cascade.detectMultiScale(face_gray, scaleFactor=2.1, minNeighbors=20)
        
        for (sx, sy, sw, sh) in smile_rect:
            x1 = int(x+w/7)
            y1 = int(y-h/10)
#             cv2.rectangle(face_color, (sx,sy), (sx+sw, sy+sh), (0,0,255), thickness=1)
#             cv2.putText(img, "Thank You", (x1, y1), 0, 1, (255,255,255), 2)

            if faces_dict == {}:
                faces_dict[count] = {}
                faces_dict[count]['face'] = img[y:y+h, x:x+w]
                faces_dict[count]['vgg'] = make_vgg(img[y:y+h, x:x+w])
                faces_dict[count]['time'] = 0
                faces_dict[count]['start_time'] = time.time()
                
                count+=1
                face_smile = img[y:y+h, x:x+w]
                cv2.imwrite("frame%d.jpg" % count, face_color)
                
#                 cv2.putText(face_color, str(count), (10,50), 0, 1, (255,255,255), 2)
#                 cv2.imshow('You', face_color)

#                 Set the position of the image window
#                 cv2.moveWindow('You', 800, 0)
                
            
            else:
                unique = True
                vgg_frame = make_vgg(img[y:y+h, x:x+w])
                for i in faces_dict:
                    if(compare(faces_dict[i]['vgg'], vgg_frame)):
                        unique = False
                        faces_dict[i]['time'] = time.time() - faces_dict[i]['start_time']
                        break
                        
                if unique:
                    faces_dict[count] = {}
                    faces_dict[count]['face'] = img[y:y+h, x:x+w]
                    faces_dict[count]['vgg'] = vgg_frame
                    faces_dict[count]['time'] = 0
                    faces_dict[count]['start_time'] = time.time()
                    
                    
                    count+=1
                    face_smile = img[y:y+h, x:x+w]
                    cv2.imwrite("frame%d.jpg" % count, face_color) 
                    
#                     cv2.putText(face_color, str(count), (10,50), 0, 1, (255,255,255), 2)
#                     cv2.imshow('You', face_color)
                    
                print(compare2(faces_dict[0]['vgg'], vgg_frame))
                print(count)
                    
    threading.Thread(target=count_frame).start()
                        
    key = cv2.waitKey(1)

    if key == 27 or off == True:
        break
    
    stop_time = time.time()

cap.release()
cv2.destroyAllWindows()

except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except

In [13]:
face_color

array([[[168, 180, 175],
        [169, 180, 177],
        [168, 180, 175],
        ...,
        [167, 170, 162],
        [167, 170, 161],
        [165, 168, 159]],

       [[165, 177, 171],
        [165, 177, 171],
        [165, 177, 171],
        ...,
        [166, 169, 160],
        [166, 169, 160],
        [166, 169, 160]],

       [[156, 168, 162],
        [157, 168, 162],
        [156, 166, 160],
        ...,
        [166, 169, 160],
        [166, 169, 160],
        [166, 169, 160]],

       ...,

       [[125, 115, 104],
        [133, 122, 112],
        [168, 157, 150],
        ...,
        [ 53,  41,  89],
        [ 55,  43,  91],
        [ 55,  44,  92]],

       [[126, 116, 110],
        [144, 133, 130],
        [173, 163, 163],
        ...,
        [ 54,  42,  88],
        [ 54,  43,  89],
        [ 56,  45,  91]],

       [[123, 115, 109],
        [146, 136, 136],
        [177, 165, 168],
        ...,
        [ 54,  44,  90],
        [ 55,  45,  91],
        [ 56,  47,  93]]

In [14]:
face_color = cv2.resize(face_color, (300,300))
print(face_color)

[[[168 180 175]
  [169 180 176]
  [168 180 175]
  ...
  [167 170 161]
  [167 170 160]
  [165 168 159]]

 [[165 178 172]
  [166 178 172]
  [166 178 172]
  ...
  [166 169 160]
  [166 169 160]
  [166 169 160]]

 [[159 171 165]
  [159 171 165]
  [159 170 164]
  ...
  [166 169 160]
  [166 169 160]
  [166 169 160]]

 ...

 [[125 115 106]
  [134 123 116]
  [159 148 142]
  ...
  [ 54  42  89]
  [ 55  43  91]
  [ 55  44  92]]

 [[125 116 110]
  [141 130 127]
  [164 154 153]
  ...
  [ 54  43  89]
  [ 55  44  90]
  [ 56  45  91]]

 [[123 115 109]
  [141 132 130]
  [167 155 157]
  ...
  [ 54  44  90]
  [ 55  45  91]
  [ 56  47  93]]]
